## Imports and Installs

In [1]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
import picos, random, gensim, nltk, re, json, os, tqdm
import cvxopt as cvx
import numpy as np
import pandas as pd
import networkx as nx
import plotly.graph_objects as go
from statistics import mode, mean
from picos import SymmetricVariable
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk import WordNetLemmatizer
from cvxopt.base import matrix as cvx_matrix
from typing import Dict, Tuple, Sequence, Any, List, Set, Union, Collection, Optional
from sklearn.cluster import KMeans
from sklearn import metrics
from statistics import mean, stdev

## Defined Functions

### Algorithms

In [2]:
def maxcut(G):
  weight = 'weight'
  num_nodes = G.number_of_nodes()
  maxcut = picos.Problem()
  X = SymmetricVariable('X', (num_nodes, num_nodes))
  LL = 1 / 4. * nx.laplacian_matrix(G, weight=weight).todense()
  L = picos.Constant('L', LL)
  maxcut.add_constraint(picos.maindiag(X) == 1)
  maxcut.add_constraint(X >> 0)
  maxcut.set_objective('max', L | X)
  maxcut.solve(verbosity=0, solver='cvxopt')

  V = X.value
  eigenvalues = np.linalg.eigvalsh(V)
  min_eigenval = min(eigenvalues)
  if np.sign(min_eigenval) == -1:
      V += (abs(min_eigenval) + 1e-4) * np.identity(V.size[0])
  V = cvx_matrix(np.linalg.cholesky(V))

  count = 0
  obj_sdp = maxcut.value
  obj = 0
  lower_bound = 0.878 * obj_sdp
  x_cut: cvx.matrix = None
  while (count < 100) or (obj < lower_bound):
      r = cvx.normal(num_nodes, 1)
      x = cvx.matrix(np.sign(V * r))
      o = (x.T * L * x).value
      if o > obj:
          x_cut = x
          obj = o
      count += 1
  x = x_cut

  indexed_nodes = list(G.nodes)
  cut_nodes = [indexed_nodes[i] for i in range(num_nodes) if x[i] < 0]
  embeddings = {indexed_nodes[i]: V[i, :] for i in range(num_nodes)}
  return cut_nodes, embeddings

In [3]:
def km_run(emb):
  km = KMeans(n_clusters = 2, n_init = 10)
  km.fit(emb)

  return km.labels_

### Label Propagation

In [4]:
def unconnected_only(nodes, G, labeled):
    unconnected = True
    for node in nodes:
        comp = nx.node_connected_component(G, node)
        if len(comp - labeled) != len(comp):
            unconnected = False
    return unconnected

In [5]:
def naive_prop_dual(idxes, color_preds, G):
  opp_prop = {}
  agg_prop = {}
  for i in range(len(idxes)):
    id = idxes[i]
    lab = color_preds[i]
    opp_prop[id] = lab
    agg_prop[id] = lab

  labeled = set(opp_prop.keys())
  unlabeled = set(G.nodes()) - labeled
  while (len(unlabeled) != 0) and (not unconnected_only(unlabeled, G, labeled)):
    n_lab = False
    while n_lab == False:
      poss = list(unlabeled)
      chosen = random.choice(poss)
      neighbors = [n for n in G.neighbors(chosen)]
      lab_n = [n for n in neighbors if n in labeled]
      if len(lab_n) > 0:
        n_lab = True
    labs = [opp_prop[n] for n in lab_n]
    if mode(labs) == 1:
        lab = 2
    else:
        lab = 1
    opp_prop[chosen] = lab
    labs = [agg_prop[n] for n in lab_n]
    if mode(labs) == 1:
        lab = 1
    else:
        lab = 2
    agg_prop[chosen] = lab
    unlabeled.remove(chosen)
    labeled.add(chosen)
  
  for node in unlabeled:
    opp_prop[node] = random.randint(1,2)
    agg_prop[node] = random.randint(1,2)

  y_opp = []
  y_agg = []
  for node in sorted(opp_prop.keys()):
    y_opp.append(opp_prop[node])
    y_agg.append(agg_prop[node])
  
  return y_opp, y_agg

### Run Batch Tests

In [6]:
def batch_maxcut(G, y_dict):
    G.remove_edges_from(nx.selfloop_edges(G))
    G.remove_nodes_from(list(nx.isolates(G)))
    comps = sorted(nx.connected_components(G), key=len, reverse=True)
    res = []
    full_preds = {}

    for comp in comps:
        G0 = G.subgraph(comp)
        k = 2
        k_core = nx.k_core(G0, k=k)
        while len(k_core) > 200:
            k += 1
            k_core = nx.k_core(G0, k=k)
        if len(k_core) == 0:
            k_core = G0
            print('No 2-core, using graph.')
            no_k = True
        else:
            print('Using', str(k) + '-core.')
            no_k = False
        cut_nodes, embeddings = maxcut(k_core)
        idxes = sorted(list(embeddings.keys()))

        preds = []
        y_true = []
        for node in sorted(k_core.nodes()):
            if node in cut_nodes:
                preds.append(2)
            else:
                preds.append(1)
            y_true.append(y_dict[node])

        preds_inv = [1 if i == 2 else 2 for i in preds]

        acc1 = metrics.accuracy_score(y_true, preds)
        acc2 = metrics.accuracy_score(y_true, preds_inv)
        accc = max(acc1, acc2)

        print('Accuracy on core :', accc)

        if not no_k:
            if acc2 > acc1:
                color_preds = preds_inv
            else:
                color_preds = preds

            full_y = []
            for node in sorted(G0.nodes()):
                if node in y_dict.keys():
                    full_y.append(y_dict[node])
                else:
                    full_y.append(1)

            opp_prop, agg_prop = naive_prop_dual(idxes, color_preds, G0)
            acco = metrics.accuracy_score(full_y, opp_prop)
            acca = metrics.accuracy_score(full_y, agg_prop)

            if acco > acca:
                accc = acco
                best = 'opposing'
            else:
                accc = acca
                best = 'agreeing'

            print('Accuracy with', best, 'propagation on', G0.number_of_nodes(), 'nodes :', accc)

            if acco > acca:
                full_prop = opp_prop
            else:
                full_prop = agg_prop
        else:
            if acc2 > acc1:
                full_prop = preds_inv
            else:
                full_prop = preds
            full_y = y_true


        nodes = sorted(G0.nodes())
        for i in range(len(nodes)):
            full_preds[nodes[i]] = full_prop[i]

        p, r, f, _ = metrics.precision_recall_fscore_support(full_y, full_prop, average='weighted')

        res.append((accc, p, r, f))
    
    return res, full_preds
        

## Run Model

In [7]:
file_path = './Processed/'
# options : 'euro', 'timme', 'cd', 'conref'
dat = 'cd'
# topics for CD; options : 'all', 'abortion', 'marijuana', 'gayRights', or 'obama'
top = 'marijuana'
# whether or not to use TIMME-All when running with TIMME; False runs TIMME-Pure
t_all = True
# the number of trials to run
trials = 10

if dat == 'cd':
  dat = dat + top

if dat == 'timme':
  if t_all:
    dat = dat + '_all'

graph_path = file_path + dat + '_graph.txt'
y_path = file_path + dat + '_ydict.json'

G = nx.read_weighted_edgelist(graph_path)
with open(y_path, 'r') as f:
  y_dict = json.load(f)

In [8]:
y_plot = []
for i in G.nodes():
    y_plot.append(y_dict[i])
data = [G, y_dict, y_plot]
graphs = [G]
conn = [" " if nx.is_connected(graph) else " not " for graph in graphs]
print("The full graph has", len(G), "nodes and is" + conn[0] + "connected.")

The full graph has 43 nodes and is not connected.


In [9]:
truth = []
res = []

for i in range(trials):
    ress, full_prop = batch_maxcut(data[0], data[1])
    avg_acc = mean([item[0] for item in ress])
    avg_f = mean([item[3] for item in ress])

    true = []
    pred = []
    for key in full_prop.keys():
        true.append(y_dict[str(key)])
        pred.append(full_prop[key])
    truth.append(true)
    res.append(pred)

Using 2-core.
Accuracy on core : 0.72
Accuracy with opposing propagation on 30 nodes : 0.7
No 2-core, using graph.
Accuracy on core : 0.8333333333333334
No 2-core, using graph.
Accuracy on core : 0.6666666666666666
No 2-core, using graph.
Accuracy on core : 1.0
No 2-core, using graph.
Accuracy on core : 1.0
Using 2-core.
Accuracy on core : 0.72
Accuracy with opposing propagation on 30 nodes : 0.7
No 2-core, using graph.
Accuracy on core : 0.8333333333333334
No 2-core, using graph.
Accuracy on core : 0.6666666666666666
No 2-core, using graph.
Accuracy on core : 1.0
No 2-core, using graph.
Accuracy on core : 1.0
Using 2-core.
Accuracy on core : 0.72
Accuracy with opposing propagation on 30 nodes : 0.7
No 2-core, using graph.
Accuracy on core : 0.8333333333333334
No 2-core, using graph.
Accuracy on core : 0.6666666666666666
No 2-core, using graph.
Accuracy on core : 1.0
No 2-core, using graph.
Accuracy on core : 1.0
Using 2-core.
Accuracy on core : 0.72
Accuracy with opposing propagation 

In [10]:
accs = []
fs = []

for i in range(len(truth)):
    acc = metrics.accuracy_score(truth[i], res[i])
    _, _, f, _ = metrics.precision_recall_fscore_support(truth[i], res[i], average='weighted')
    accs.append(acc)
    fs.append(f)

acc_m = mean(accs)
f_m = mean(fs)
if len(accs) > 1:
    acc_st = stdev(accs)
    f_st = stdev(fs)
    print("Mean Accuracy :", round(acc_m, 4) * 100, "\tSt. dev :", round(acc_st, 4) * 100)
    print("Mean Weighted F1 :", round(f_m, 4) * 100, "\tSt. dev :", round(f_st, 4) * 100)
else:
    print("Accuracy :", round(acc_m, 4) * 100)
    print("Weighted F1 :", round(f_m, 4) * 100)

Mean Accuracy : 73.95 	St. dev : 0.98
Mean Weighted F1 : 74.72 	St. dev : 0.9299999999999999
